# Quick Start: Azure AI Foundry SDK

### Step 1: Create a new agent
This step shows how to create an Azure AI Agent using the Foundry SDK, attach tools, and start a conversation thread. The code demonstrates agent creation, thread creation, sending a message, running the agent, and retrieving responses.

* [Azure AI Agents client library for Python - version 1.0.2](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-agents-readme?view=azure-python)
* [Azure AI Projects client library for Python - version 1.0.0b12](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview)

In [ ]:
# --- Standard Library Imports ---
import os
import sys
from dotenv import load_dotenv

# --- Azure AI Foundry SDK for AI agents and services ---
from azure.ai.agents.models import CodeInterpreterTool, FilePurpose, FileSearchTool, ListSortOrder
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Load the environment variables from a .env file
load_dotenv()

# Check the current working Python execution environment
print(f"Current Python executable: {sys.executable}")

Current Python executable: c:\Users\Yu-Hong (Frank) Lin\Repos\azure-ai-agent\.venv\Scripts\python.exe


In [ ]:
# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],  # Project endpoint from environment variable
    credential=DefaultAzureCredential(),      # Use Azure Default Credential for authentication
)

code_interpreter = CodeInterpreterTool()

with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
        name="01_SDK agent",                     # Name of the agent
        instructions="You are a helpful agent",     # Instructions for the agent
        tools=code_interpreter.definitions,         # Attach the tool
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")
    
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",                                      # Role of the message sender
        content="What is the weather in Seattle today?",  # Message content
    )
    print(f"Created message, ID: {message['id']}")
    
    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")
    
    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
    # Fetch and log all messages
    messages = project_client.agents.messages.list(thread_id=thread.id)
    for message in messages:
        print(f"Role: {message.role}, Content: {message.content}")

### 02 Listing AI Project Agents
This step demonstrates how to list all agents in your Azure AI Project using the Foundry SDK. The code retrieves and prints all available agents.

In [ ]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  
)

# List the agents
with project_client:
    
    agent_list = project_client.agents.list_agents()
    for _agent in agent_list:
        print(_agent)

### 03 Deleting an Agent from AI Project Client
This step shows how to delete an existing agent from your Azure AI Project using the Foundry SDK. The code demonstrates the deletion process.

In [ ]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),  
)

# Delete the agent when done
with project_client:
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

---

##### Reference: [Create a new Azure AI Foundry Agent Service project - Azure AI Foundry | Microsoft Learn](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/quickstart?pivots=programming-language-python-azure)